In [12]:
!pip install colabcode 
!pip install fastapi    


     |████████████████████████████████| 45 kB 1.5 MB/s 
     |████████████████████████████████| 745 kB 7.6 MB/s 
     |████████████████████████████████| 8.3 MB 11.8 MB/s 
     |████████████████████████████████| 428 kB 49.6 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 392 kB 40.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 112 kB 37.7 MB/s 
     |████████████████████████████████| 68 kB 5.7 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 56 kB 3.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-py3-none-any.whl size=19262 sha256=217469c43063abc239de4d55eb4508d959a1cfdc8c9f1980be64156f3bd5bc34
  Stored in directory: /root/.cache/pip/wheels/1b/f7/72/35c95a53d15b91dd00df6cf1304d49a31ec5ed6f954c2d4e32
Successfully built pyngrok
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninst

     |████████████████████████████████| 51 kB 304 kB/s 
     |████████████████████████████████| 10.1 MB 8.6 MB/s 
     |████████████████████████████████| 60 kB 6.4 MB/s 


In [13]:
from colabcode import ColabCode       # A Python package that allows you to start a code server right from your Colab notebooks without setting up anything locally on your system
from fastapi import FastAPI     
cc  = ColabCode(port=1200,code=False) #setting up the server


In [1]:
from google.colab import drive
drive.mount('/content/drive')        #mounting the drive to load the required files from drive

Mounted at /content/drive


In [14]:
import tensorflow as tf
import keras              
import pickle
import pandas as pd 
import tensorflow as tf       
from tensorflow.keras.preprocessing.sequence import pad_sequences                                 #helps in regularising the lengths of sequences 
tokenizer = pickle.load(open("/content/drive/MyDrive/ENTHIRE/GRU_Model/tokenizer_file2.pkl","rb"))  #load the tokenizer file 
model = keras.models.load_model("/content/drive/MyDrive/ENTHIRE/GRU_Model/29_model")               #loads the already saved model


In [15]:
#function to clean the inputs text by removing unwanted characters eg:(@,#,^,*)
!pip install emoji
import re
import emoji
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def process_text(text):  
    new_text = text.lower()                                  #making all the characters lower case 
    new_text = re.sub(r'@\w+', '', new_text)                 # Remove @s
    new_text = re.sub(r'#', '', new_text)                    # Remove hashtags
    new_text = re.sub(r':', ' ', emoji.demojize(new_text))   # Turn emojis into words
    new_text = re.sub(r'http\S+', '',new_text)               # Remove URLs
    new_text = re.sub(r'\$\S+', 'dollar', new_text)          # Change dollar amounts to dollar
    new_text = re.sub(r'[^a-z0-9\s]', '', new_text)          # Remove punctuation
    new_text = re.sub(r'[0-9]+', 'number', new_text)         # Change number values to number
    new_text = new_text.split(" ")                           # splits the text into a list of strings after breaking the given text by the specified separator in our case it is " ".
    new_text = list(map(lambda x: ps.stem(x), new_text))     # Stemming the words
    new_text = list(map(lambda x: x.strip(), new_text))      # Stripping whitespace from the words
    if '' in new_text:
        new_text.remove('')
    return new_text

In [16]:
app = FastAPI()                                                          #setting up the app
@app.get("/")
async def index():
    return {"message":"hey, have a nice day "}


@app.get("/{data}")                                                      #get function to get the string input from the user 
def predict_sentiment(data:str):
    text_list = [process_text(data)]                                     #processing the text to remove unwanted characters
    testing_sequences = tokenizer.texts_to_sequences(text_list)          #coverts the text to sequence of numbers by using the tokenizer function
    padded = pad_sequences(testing_sequences, maxlen=90, padding='post') #pads the sequence with zeros at the end to regularise the length of all the sequences to the maximum sequence length
    prediction = model.predict(padded)                                   #feeding the padded sequence to the model for it give a prediction
    if prediction[0][0]>=0.5:                                            #we consider prediction as positive if its value is greater than or equal to  0.5
        prediction = "Positive"
    else:
        prediction = "Negative"                                          #we consider prediction as negative if its value is lesser than or equal to  0.5
    return {
        "prediction" : prediction
    }


In [17]:
cc.run_app(app)                                                        #running the app ...

Public URL: NgrokTunnel: "https://a4742912e4b3.ngrok.io" -> "http://localhost:1200"


INFO:     Started server process [70]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:1200 (Press CTRL+C to quit)


INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     49.44.83.137:0 - "GET / HTTP/1.1" 200 OK
INFO:     49.44.82.168:0 - "GET /hi%20%40airlines%20your%20flights%20are%20the%20worst HTTP/1.1" 200 OK
INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET /hi%20%40airlines%20your%20flights%20are%20the%20worst HTTP/1.1" 200 OK
INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET /hi%20i%20had%20the%20worst%20experience%20in%20my%20life HTTP/1.1" 200 OK
INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     49.44.81.197:0 - "GET /hi%20i%20had%20the%20worst%20experience%20in%20my%20life HTTP/1.1" 200 OK
INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4072:297:9479:44bd:aec3:b318:6515:0 - "GET

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [70]
